In [1]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import sys

In [2]:
# Import reviews to find the hostels users liked
revs_df = pd.read_csv('/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/clean-data/clean_revs_oct7.csv')
revs_df.head(2)

,Unnamed: 0,text,user_id,rev_date,score,hostel,location,link,country,city,day,month,year,country_id,hostel_id
0,0,excellent value,1189474,30th Jun 2004,7.7,Fortuna Botel,"Budapest, Hungary",http://www.hostelworld.com/hosteldetails.php/F...,Hungary,Budapest,30,6,2004,28.0,0
1,1,i had a great time there because of the beach ...,1595059,23rd Dec 2004,7.7,Equity Point Sea,"Barcelona, Spain",http://www.hostelworld.com/hosteldetails.php/E...,Spain,Barcelona,23,12,2004,5.0,1


In [3]:
# find reviews with scores above a 7.5
X = revs_df[revs_df['score']>7.5]

# save every users hostels reviewed above a 7.5 to lists
ids = []
hostels1 = []
countries1 = []
for id in revs_df['user_id'].unique():
    ids.append(id)
    hostels1.append(list(X['hostel'][X['user_id'] == id]))
    countries1.append(list(revs_df['country'][revs_df['user_id'] == id]))
len(ids),len(hostels1), len(countries1)

(10929, 10929, 10929)

In [4]:
# create a df with users and hostels liked
user_travels = pd.DataFrame()
user_travels['user_id'] = ids
user_travels['hsts_liked'] = hostels1
user_travels['countries'] = countries1

user_travels.head(3)

,user_id,hsts_liked,countries
0,1189474,[Fortuna Botel],[Hungary]
1,1595059,[Equity Point Sea],[Spain]
2,3637809,[The Naughty Squirrel Backpackers Hostel],[Latvia]


In [5]:
# import users information
users_df = pd.read_csv('/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/clean-data/clean_users_oct7.csv')
del users_df['Unnamed: 0']
del users_df['num_revs']
del users_df['country_id']
users_df.head(2)

,user_id,nationality,group,age
0,1189474,England,NaN,not specified
1,1595059,Hong Kong,NaN,not specified


In [6]:
# import the topic prob for users 
users_topics = pd.read_csv('/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/clean-data/user_rev_info_4dist_oct7.csv')
del users_topics['Unnamed: 0']
del users_topics['num_revs']
del users_topics['country_id']
del users_topics['group_id']
del users_topics['age_id']
users_topics.head(2)

,user_id,score,Rt1,Rt2,Rt3,Rt4,Rt5,Rt6,Rt7,Rt8,Rt9,Rt10,num_hostels
0,1004454,8.3,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.899995,0.011112,0.011112,1
1,1004711,8.9,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000,1


In [7]:
# make a list of topics users have a high prob in 
top_topics = []
for i,u in enumerate(users_topics['user_id']):
    topics = []
    if users_topics['Rt1'][i] >= 0.3:
        topics.append('Rt1')
    if users_topics['Rt2'][i] >= .3:
        topics.append('Rt2')
    if users_topics['Rt3'][i] >=.3:
        topics.append('Rt3')
    if users_topics['Rt4'][i] >=.3:
        topics.append('Rt4')
    if users_topics['Rt5'][i] >=.3:
        topics.append('Rt5')
    if users_topics['Rt6'][i] >=.3:
        topics.append('Rt6')
    if users_topics['Rt7'][i] >=.3:
        topics.append('Rt7')
    if users_topics['Rt8'][i] >=.3:
        topics.append('Rt8')
    if users_topics['Rt9'][i] >=.3:
        topics.append('Rt9')
    if users_topics['Rt10'][i] >=.3:
        topics.append('Rt10')
    top_topics.append(topics)
    
users_topics['topics'] = top_topics
users_topics.head(2)

,user_id,score,Rt1,Rt2,Rt3,Rt4,Rt5,Rt6,Rt7,Rt8,Rt9,Rt10,num_hostels,topics
0,1004454,8.3,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.899995,0.011112,0.011112,1,[Rt8]
1,1004711,8.9,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000,1,[Rt8]


In [8]:
# put socres in groups
def score(value):
    if value >8.5:
        return '>8.5'
    elif value > 7:
        return '7-8.5'
    elif value >5:
        return '5-7'
    else:
        return '<5'

users_topics['score'] = users_topics['score'].map(score)
users_topics.head(2)

,user_id,score,Rt1,Rt2,Rt3,Rt4,Rt5,Rt6,Rt7,Rt8,Rt9,Rt10,num_hostels,topics
0,1004454,7-8.5,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.899995,0.011112,0.011112,1,[Rt8]
1,1004711,>8.5,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000,1,[Rt8]


In [9]:
# add 'hs' for hostels to the num_hostels 
def hsts(value):
    return 'Hs:'+ str(value)

users_topics['num_hostels'] = users_topics['num_hostels'].map(hsts)
users_topics.head(2)

,user_id,score,Rt1,Rt2,Rt3,Rt4,Rt5,Rt6,Rt7,Rt8,Rt9,Rt10,num_hostels,topics
0,1004454,7-8.5,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.899995,0.011112,0.011112,Hs:1,[Rt8]
1,1004711,>8.5,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000,Hs:1,[Rt8]


In [10]:
X = users_topics[['user_id','score','num_hostels','topics']]
X.shape, user_travels.shape

((10929, 4), (10929, 3))

In [11]:
# merge X from user topics with user_travels df on user id
users = pd.merge(user_travels, X, on='user_id')
users.head(3)

,user_id,hsts_liked,countries,score,num_hostels,topics
0,1189474,[Fortuna Botel],[Hungary],7-8.5,Hs:1,[Rt6]
1,1595059,[Equity Point Sea],[Spain],7-8.5,Hs:1,[Rt2]
2,3637809,[The Naughty Squirrel Backpackers Hostel],[Latvia],>8.5,Hs:1,[]


In [12]:
# edit use rnationality so it says nationality in the values
def nat(value):
    return 'nationality: '+ str(value)

users_df['nationality'] = users_df['nationality'].map(nat)
users_df.head(3)

,user_id,nationality,group,age
0,1189474,nationality: England,NaN,not specified
1,1595059,nationality: Hong Kong,NaN,not specified
2,3637809,nationality: Australia,Male,31-40


In [13]:
# merge users df with user info df 
Users = pd.merge(users_df, users, on='user_id')
Users.head(3)

,user_id,nationality,group,age,hsts_liked,countries,score,num_hostels,topics
0,1189474,nationality: England,NaN,not specified,[Fortuna Botel],[Hungary],7-8.5,Hs:1,[Rt6]
1,1595059,nationality: Hong Kong,NaN,not specified,[Equity Point Sea],[Spain],7-8.5,Hs:1,[Rt2]
2,3637809,nationality: Australia,Male,31-40,[The Naughty Squirrel Backpackers Hostel],[Latvia],>8.5,Hs:1,[]


In [14]:
# make a column that is a list of the users information
users_stuff = []
for i,u in enumerate(Users['user_id']):
    u_info =[]
    for h in Users['hsts_liked'][i]: 
        u_info.append(h)
    for c in Users['countries'][i]:
        u_info.append(c)
    u_info.append(Users['score'][i])
    u_info.append(Users['num_hostels'][i])
    for t in Users['topics'][i]:
        u_info.append(t)
    u_info.append(Users['nationality'][i])
    u_info.append(Users['group'][i])
    u_info.append(Users['age'][i])
    users_stuff.append(u_info)
    
# add new column to Users df 
Users['user_stuff'] = users_stuff

In [15]:
# put info into a df for finding distance
user_dist = Users[['user_id','user_stuff','hsts_liked']]
user_dist.head(3)

,user_id,user_stuff,hsts_liked
0,1189474,"[Fortuna Botel, Hungary, 7-8.5, Hs:1, Rt6, nat...",[Fortuna Botel]
1,1595059,"[Equity Point Sea, Spain, 7-8.5, Hs:1, Rt2, na...",[Equity Point Sea]
2,3637809,"[The Naughty Squirrel Backpackers Hostel, Latv...",[The Naughty Squirrel Backpackers Hostel]


In [20]:
user_dist.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/clean-data/user_dist_oct10.csv',encoding='utf8')

In [16]:
# equation for finding jaccard distance between users 
def j_dist(user):
    distance = []
    users1 = []
    hostels = []
    user_a = set(user_dist ['user_stuff'][user_dist.user_id == user].values[0])
    for i,id in enumerate(user_dist.user_id):
        user_b = set(user_dist.user_stuff[i])

        numerator = len(user_a.intersection(user_b)) * 1.0 # * 1.0 to cast as a float
        denominator = len(user_a.union(user_b)) * 1.0
        distance.append(numerator / denominator)
        users1.append(id)
        hostels.append(user_dist['hsts_liked'][i])
    distances = pd.DataFrame()
    distances['users'] = users1
    distances['distances']=distance
    distances['hostels'] = hostels
    return distances.sort_values('distances', ascending = False)[1:5]

In [17]:
j_dist(3298399)

,users,distances,hostels
6563,8144810,0.312500,[Iva Apartments]
3767,9802505,0.300000,"[El Jardin Hostal, Way Hostel, Hostel Rosemary]"
9161,9936860,0.277778,"[Boipeba Bed and Breakfast, Hostal Andalucia]"
3425,8211885,0.250000,[Bed & Bike Barcelona]
